In [258]:
using JuMP
using Gurobi
using CSV
using DataFrames

In [259]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06


### Data importation

In [260]:
centers = CSV.File("HW3 data/centers.csv",header=0) |> Tables.matrix;
stations = CSV.File("HW3 data/stations.csv",header=0) |> Tables.matrix;
landfills = CSV.File("HW3 data/landfills.csv",header=0) |> Tables.matrix;
q = CSV.File("HW3 data/q.csv",header=0) |> Tables.matrix;

centers2 = CSV.File("HW3 data/centers2.csv",header=0) |> Tables.matrix;
stations2 = CSV.File("HW3 data/stations2.csv",header=0) |> Tables.matrix;
landfills2 = CSV.File("HW3 data/landfills2.csv",header=0) |> Tables.matrix;
q2 = CSV.File("HW3 data/q2.csv",header=0) |> Tables.matrix;

centers_all = [centers;centers2];
stations_all = [stations;stations2];
landfills_all = [landfills;landfills2];
q_all = [q;q2];

n1 = size(centers)[1]
s1 = size(stations)[1]
m1 = size(landfills)[1]
n2 = size(centers2)[1]
s2 = size(stations2)[1]
m2 = size(landfills2)[1]
n_all = n1+n2;
s_all = s1+s2;
m_all = m1+m2;

### Part A

Choose which 5 of 15 landfills to build to minimize transportation costs of driving (distances from centers to landfills)

In [261]:
k = 5 # number of landfills to build

# Calculate distance matrix
distances = [sqrt((centers[i, 1] - landfills[j, 1])^2 + (centers[i, 2] - landfills[j, 2])^2) for i in 1:n1, j in 1:m1]

model = Model(Gurobi.Optimizer)

# Decision Variables
@variable(model, x[1:m1], Bin)   # 1 if landfill j is selected, 0 otherwise
@variable(model, y[1:n1, 1:m1] >= 0)  # amount of waste sent from center i to landfill j

# Objective: Minimize transportation cost
@objective(model, Min, sum(distances[i, j] * y[i, j] for i in 1:n1, j in 1:m1))

# Constraints
@constraint(model, sum(x) == k)  # Select exactly k landfills
# Meet waste requirements at each center (waste has to go to built landfill)
@constraint(model, [i=1:n1], sum(y[i, j] * x[j] for j in 1:m1) == q[i])  

optimize!(model)

# Results
selected_landfills = [j for j in 1:m1 if value(x[j]) > 0.5]
total_distance = objective_value(model)
println("Selected Landfills: ", selected_landfills)
println("Total Distance Traveled: ", total_distance)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 765 columns and 15 nonzeros
Model fingerprint: 0x6535e513
Model has 50 quadratic constraints
Variable types: 750 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [2e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+00]
  QRHS range       [5e+02, 2e+03]
Presolve time: 0.00s
Presolved: 1551 rows, 3015 columns, 4515 nonzeros
Presolved model has 1500 SOS constraint(s)
Variable types: 2250 continuous, 765 integer (765 binary)
Found heuristic solution: objective 1177706.7451

Root relaxation: objective 8.404875e+05, 146 iterations, 0.00 seconds (0.00 work units)

  

### Parts B and C - Transfer Stations Added

## Part B - Find optimal 5 locations given that waste can optionally be compacted at stations in stations.csv

f(5, 0)

## Part C - Adding neighboring region
**Part 1, same as part B but using centers2, landfils2, stations2**

f(5, 1)

**Part 2, combining both regions**

f(10, 2)

In [262]:
"""
f(number_landfills, neighbor)
num_landfills - int representing how many landfills to optimize for
neighbor - selection of which problem to solve using the vals listed below:
    0 - original problem for landfills in x = [0,100]
    1 - neighboring region for landfills in x = [100,200]
    2 - both regions combined
"""
function f(num_landfills::Int=5, neighbor::Int=0)
    cent = centers
    land = landfills
    stat = stations
    trash_amnts = q
    s = s1
    n = n1
    m = m1
    if neighbor == 1 # only conseder neighboring area
        cent = centers2
        land = landfills2
        stat = stations2
        trash_amnts = q2
        s = s2
        n = n2
        m = m2
    elseif neighbor == 2 # both areas
        cent = centers_all
        land = landfills_all
        stat = stations_all
        trash_amnts = q_all
        s = s_all
        n = n_all
        m = m_all
    else
    end
    model = Model(Gurobi.Optimizer)

    # Calculate distance matrices
    distances = [sqrt((cent[i, 1] - land[j, 1])^2 + (cent[i, 2] - land[j, 2])^2) for i in 1:n, j in 1:m]
    distances_center_to_stations = [sqrt((cent[i, 1] - stat[k, 1])^2 + (cent[i, 2] - stat[k, 2])^2)  for i in 1:n, k in 1:s]
    distances_stations_to_landfils = [sqrt((stat[k, 1]- land[j, 1])^2 + (stat[k, 2]- land[j, 2])^2) for k in 1:s, j in 1:m]

    # Additional Costs
    station_cost = 10000  # daily operating cost per transfer station
    compact_cost = 0.50  # per mile-ton cost for compacted waste
    uncomp_cost = 1.0  # per mile-ton cost for uncompacted waste

    # Decision Variables from Part A
    @variable(model, x[1:m], Bin)   # 1 if landfill j is selected, 0 otherwise
    @variable(model, y[1:n, 1:m] >= 0)  # amount of waste sent from center i to landfill j

    # Additional Decision Variables
    @variable(model, z[1:s], Bin)  # 1 if transfer station is built at location k
    @variable(model, w[1:n, 1:s] >= 0)  # waste sent from center i to transfer station k as uncompacted waste
    @variable(model, t[1:s, 1:m] >= 0)  # waste sent from transfer station k to landfill i as compacted waste

    # new objective: sum of cost of not compressing + cost of traveling to be compressed + cost of traveling after compressed + cost of station maintinance
    @objective(model, Min, sum(distances[i, j] * y[i, j] * uncomp_cost for i in 1:n, j in 1:m) +  # uncompressed distance cost
                            sum(distances_center_to_stations[i, k] * w[i, k] * uncomp_cost for i in 1:n, k in 1:s) +  # uncompressed distance cost, on way to station
                            sum(distances_stations_to_landfils[k, j] * t[k, j] * compact_cost for k in 1:s, j in 1:m) +   # compressed distance cost, on way to landfill
                            sum(station_cost * z[k] for k in 1:s)) # station cost

    # Original Constraints - Part A
    @constraint(model, sum(x) == num_landfills)  # Select exactly k landfills

    # Additional Constraints

    # Waste from each center must be entirely taken away (either to landfill or transfer station)
    @constraint(model, [i=1:n], sum(y[i, j] for j in 1:m) + sum(w[i, k] for k in 1:s) == trash_amnts[i])
    # Transfer station capacity, cannot send more than 2000 tons of trash to (valid) station
    @constraint(model, [k=1:s], sum(w[i, k] for i in 1:n) <= 2000 * z[k])
    # No lingering trash in transfer stations at the end of the day
    @constraint(model, [k=1:s], sum(w[i, k] for i in 1:n) == sum(t[k, i] for i in 1:m))

    # verify that all waste sent directly to landfill gets sent to valid landfill 
    @constraint(model, [i=1:n, j=1:m], y[i,j] <= trash_amnts[i] * x[j])
    # verify that all waste first sent to transfer station gets sent to valid lanfil
    @constraint(model, [k=1:s, j=1:m], t[k,j] <= 2000 * x[j])
 
    optimize!(model)

    # Results
    selected_landfills = [j for j in 1:m if value(x[j]) == 1]
    total_cost = objective_value(model)
    selected_transfer_stations = [j for j in 1:s if value(z[j]) == 1]

    # Calculate sum of all possible distances the trash could have taken
    distance_c_s = sum(value(distances_center_to_stations[i, k] * value(w[i, k])) for i in 1:n, k in 1:s if value(z[k]) == 1) # ton miles from centers to sations
    distances_s_l = sum(value(distances_stations_to_landfils[k, j]) * value(t[k, j]) for k in 1:s, j in 1:m if value(z[k]) == 1) # ton miles from stations to landfills
    ditances_none =  sum(value(distances[i, j]) * value(y[i, j]) for i in 1:n, j in 1:m) # distances straight from centers to landfills
    distance_traveled_by_waste = distance_c_s + distances_s_l + ditances_none

    println("Selected Landfills: ", selected_landfills)
    println("Total Cost: ", total_cost)
    println("Transportation Cost: ", distance_c_s + ditances_none + (0.5*distances_s_l))
    println("Maintenance Cost: ", sum(value(z[k]) * station_cost for k in 1:s))
    println("Selected Transfer Stations: ", selected_transfer_stations)
    println("Distance Traveled by Waste: ", distance_traveled_by_waste)

    return total_cost
end

f

### Funciton Calls to solve B, C

In [263]:
f(5, 0) # part B

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1651 rows, 4065 columns and 12065 nonzeros
Model fingerprint: 0x63457002
Variable types: 4000 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+03]
Found heuristic solution: objective 2401147.9666
Presolve time: 0.01s
Presolved: 1651 rows, 4065 columns, 12065 nonzeros
Variable types: 4000 continuous, 65 integer (65 binary)

Root relaxation: objective 7.466651e+05, 339 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/N

808642.7541913443

In [264]:
f(5, 1) # part C, neighboring region only problem

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1491 rows, 3415 columns and 10115 nonzeros
Model fingerprint: 0xb0c3d1fb
Variable types: 3350 continuous, 65 integer (65 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e-01, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 3e+03]
Found heuristic solution: objective 3262000.1728
Presolve time: 0.01s
Presolved: 1491 rows, 3415 columns, 10115 nonzeros
Variable types: 3350 continuous, 65 integer (65 binary)

Root relaxation: objective 1.074031e+06, 409 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/N

1.1517612304605765e6

In [265]:
f(10, 2) # part C, both regions combined

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 24.0.0 24A348)

CPU model: Apple M2 Max
Thread count: 12 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 5991 rows, 14830 columns and 44230 nonzeros
Model fingerprint: 0x50779187
Variable types: 14700 continuous, 130 integer (130 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e-01, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 3e+03]
Found heuristic solution: objective 9493590.3783
Presolve time: 0.02s
Presolved: 5991 rows, 14830 columns, 44230 nonzeros
Variable types: 14700 continuous, 130 integer (130 binary)

Root relaxation: objective 1.720419e+06, 762 iterations, 0.02 seconds (0.05 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Ga

1.865384503441201e6

# Calculating Improvements by joining the two regions together

In [266]:
combined_costs = 808642.7541913443 + 1.1517612304605765e6

money_saved = combined_costs - 1.865384503441201e6
println("Saved ", money_saved, " in daily costs")

# waste generated in each region
waste_1 = sum(q)
waste_2 = sum(q2)
println("Region 1 produces ", waste_1, " tons of waste daily")
println("Region 2 produces ", waste_2, " tons of waste daily")

total_waste = waste_1+waste_2
reg1_cost = (waste_1/(waste_1+waste_2)) * 1.6909306929608872e6
reg2_cost = (waste_2/(waste_1+waste_2)) * 1.6909306929608872e6
println("Combined, the produce ", total_waste, " tons of trash daily, meaning region 1 pays ", reg1_cost, " of the daily costs and region 2 pays ", reg2_cost, " of the daily costs")
println("Region 1 saves ", reg1_cost-76972.55807377936, " and region 2 saves ", reg2_cost - 1.0270328487382131e6, " in daily costs")

Saved 95019.4812107198 in daily costs
Region 1 produces 48803.17 tons of waste daily
Region 2 produces 78624.9 tons of waste daily
Combined, the produce 127428.06999999999 tons of trash daily, meaning region 1 pays 647602.8246114689 of the daily costs and region 2 pays 1.0433278683494184e6 of the daily costs
Region 1 saves 570630.2665376896 and region 2 saves 16295.019611205324 in daily costs
